In [1]:
%matplotlib notebook
from __future__ import division

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, ylabel, xlabel, yscale, xscale, legend, subplots, gca, gcf, subplots, figure
from matplotlib.ticker import FormatStrFormatter
from theano import function
import numpy as np
import gzip
import cPickle
from scipy.optimize import minimize
from climin.util import optimizer
from itertools import repeat, cycle, islice, izip, imap, islice
import random
import time
from pprint import pprint
inf = float("inf")

gnumpy: failed to import cudamat. Using npmat instead. No GPU will be used.


In [3]:
from breze.learn.data import one_hot
from breze.learn.base import cast_array_to_local_type
from schlichtanders.myfunctools import compose, meanmap, meanexpmap, summap, compose_fmap, Average, as_wrapper, AverageExp
from schlichtanders import myfunctools
from schlichtanders.mygenerators import eatN, chunk, chunk_list, every, takeN
from schlichtanders.myplot import add_val, add_point
from schlichtanders.mycontextmanagers import ignored

In [4]:
from theano_models import (Merge, Flatten, Reparameterize, Center, reduce_all_identities, variational_bayes,
                           normalizing_flow,
                           fix_params, inputting_references, outputting_references)
from theano_models.tools import (as_tensor_variable, total_size, clone, clone_all,PooledRandomStreams,
                                 get_profile, squareplus, squareplus_inv, softplus, softplus_inv)
from theano_models.visualization import d3viz
from IPython.display import IFrame
import theano_models.deterministic_models as dm
import theano_models.probabilistic_models as pm
import theano_models.extra_models as em
import theano_models.postmaps as post
from theano_models.composing import normalizing_flow, variational_bayes

In [5]:
import theano
import theano.tensor as T
from theano.printing import debugprint
from theano.tensor.shared_randomstreams import RandomStreams

In [6]:
from sqlalchemy import Column, Integer, Unicode, UnicodeText, String, PickleType, Float
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [7]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [8]:
inputting_references.update(['to_be_randomized'])
inputting_references, outputting_references

({'extra_inputs',
  'flat',
  'inputs',
  'n_data',
  'parameters',
  'parameters_positive',
  'parameters_psumto1',
  'to_be_randomized'},
 {'kl_prior', 'loglikelihood', 'norm_det', 'norm_dets', 'outputs'})

# target (true)

In [9]:
data_target = Merge(pm.Gauss(2, init_var=np.exp(-2)), ignore_references={'parameters_positive'}) # fix var for likelihood
param_target = Merge(pm.Gauss(2, init_var=10), ignore_references={'parameters'}) # fix mean for parameters
# param_model = normalizing_flow(dm.LocScaleTransform(), fix_params(pm.DiagGauss(2)))

prior = fix_params(pm.Gauss(2, init_var=10))  # fix everything for the gaussian
target = variational_bayes(data_target, 'parameters', param_target, priors=prior)
# loglikelihood equals standard log-probability objective
target

variational_lower_bound { 'inputs': [],
  'kl_prior': variational_lower_bound.kl_prior,
  'logposterior': variational_lower_bound.logposterior,
  'logprior': variational_lower_bound.logprior,
  'n_data': n_data,
  'outputs': Gauss.outputs,
  'parameters': [],
  'parameters_positive': [var]}

---

In [10]:
t_sampler = target.function()
n_samples = 10000
training = np.array([t_sampler() for _ in xrange(n_samples)])
targets = training

In [11]:
target['n_data'].set_value(len(targets))

# posterior

$$p(W|D) = \frac{p(D|W)p(W)}{p(D)=\int P(D|W) P(W) dW}$$

In [12]:
pD = pm.Gauss(2, init_var=10 + np.exp(-2))

# model

In [13]:
model = target

----

In [14]:
# model = Merge(model, Reparameterize(model['parameters_positive'], squareplus, squareplus_inv))
# UncertainWeights uses softplus parameterization
_model = model
_model = Merge(_model, Reparameterize(_model['parameters_positive'], softplus, softplus_inv))
_model = Merge(_model, Center(_model['parameters']))
_model = Merge(_model, Flatten(_model['parameters']))
_model

Merge9 { 'flat': "var_softplus_centered",
  'inputs': [],
  'kl_prior': variational_lower_bound.kl_prior,
  'logposterior': variational_lower_bound.logposterior,
  'logprior': variational_lower_bound.logprior,
  'n_data': n_data,
  'outputs': Gauss.outputs,
  'parameters': [],
  'parameters_positive': []}

# Visualization

In [15]:
scatter_kwargs = dict(edgecolors="face", alpha=0.01, s=5000)
outer = dict(i=0)
example_samples = 1000
def plot_fit(fig, ax, givens={}, time_delay=0.1):
    outer['i'] += 1
    ax.clear()
    
    # targets
    ax.plot(targets[:example_samples,0], targets[:example_samples,1], 'k.')
#     ax.scatter(targets[:,0], targets[:,1], c='k', **scatter_kwargs)

    # model
    m_sampler = model.function(givens=givens)
    m_samples = np.array([m_sampler() for _ in xrange(example_samples//2)])
    ax.scatter(m_samples[:,0], m_samples[:,1], c='r',**scatter_kwargs)

    fig.canvas.draw()
    if time_delay is not None:
        time.sleep(time_delay)

TODO the above throughs error, however this would be not needed any longer if the logP are taking vectors of values instead of values as argument

In [19]:
values = np.linspace(-10, 10, 10)
Xvalues, Yvalues = np.meshgrid(values, values)
XY = np.c_[Xvalues.flat, Yvalues.flat]

logP_target = T.stack([theano.clone(target.loglikelihood['outputs'],
                             replace={target.loglikelihood['inputs'][0]:as_tensor_variable(rv, "num_rv")})
                for rv in XY])
f_logP_target = theano.function([], logP_target)

def plot_distr_target(fig):
    fig.clear()
#     Z = np.reshape(f_logP_target(), Xvalues.shape)  
#     Z = np.reshape(Average(1000)(f_logP_target), Xvalues.shape)
    # as we use likelihood, we have must use AverageExp to get the right average
    Z = np.reshape(AverageExp(1000)(f_logP_target), Xvalues.shape)        
    for i in range(2):
        ax = fig.add_subplot(1,2,i+1)
        if i==1:
            Z = np.exp(Z)
        h = ax.contourf(Xvalues, Yvalues, Z)
        plt.colorbar(h, ax=ax)
        fig.tight_layout()
        fig.canvas.draw()


flat = T.vector()
logP_model = T.stack([theano.clone(model.logP['outputs'],
                             replace={model.logP['inputs'][0]:as_tensor_variable(rv, "num_rv2")
                                     ,_model['flat']: flat})
                # for some weird reason, _model['flat'] is no longer there in the cloned version
                for rv in XY])
f_logP_model = theano.function([flat], logP_model)

        
def plot_distr_model(fig, flat):
    fig.clear()
#     Z = np.reshape(f_logP_model(flat), Xvalues.shape)
    # as we use variational lower bound, a normal average is fine by the math
    Z = np.reshape(Average(1000)(f_logP_model, flat), Xvalues.shape)        
#     Z = np.reshape(AverageExp(1000)(f_logP_model, flat), Xvalues.shape)        
    for i in range(2):
        ax = fig.add_subplot(1,2,i+1)
        if i==1:
            Z = np.exp(Z)
        h = ax.contourf(Xvalues, Yvalues, Z)
        plt.colorbar(h, ax=ax)
        fig.tight_layout()
        fig.canvas.draw()

# for i in range(20):

TypeError: Cannot convert Type TensorType(float64, vector) (of Variable <TensorType(float64, vector)>) into Type TensorType(float64, (True,)). You can try to manually convert <TensorType(float64, vector)> into a TensorType(float64, (True,)).

In [ ]:
%debug

# optimizer

In [ ]:
T.vector().broadcastable

In [ ]:
reduce_all_identities()

batch_size = 1
n_batches = len(targets) // batch_size  # after this many steps we went through the whole data set once

climin_args = izip(imap(lambda x:(x,), chunk(batch_size, cycle(training))), repeat({}))

def weights_regularizer_1epoch():
    for i in range(1, n_batches+1):
        yield 2**(n_batches - i) / (2**n_batches - 1)
        
assert len(list(weights_regularizer_1epoch())) == n_batches

In [ ]:
postmap = compose(post.flat_numericalize_postmap, post.probabilistic_optimizer_postmap) 
postmap_kwargs = {
    'pre_compile':{'num_loss': False, 'num_jacobian': False, 'num_hessian': False},
    'mapreduce': meanmap,  # TODO add more functionality for composed fmaps, with args
#     'wrapper': as_wrapper(AverageExp(40)),
#     'adapt_init_params': lambda ps: ps + np.random.normal(size=ps.size, scale=1),
#     'profile': True,
#     'mode': 'FAST_COMPILE'
}
optimizer_kwargs = postmap(_model, **postmap_kwargs)

In [ ]:
opt = optimizer(
#     identifier='adam',
    identifier="rmsprop",
    step_rate=1e-3,
#     momentum=0.9,
    args=climin_args,
    **post.climin_postmap(optimizer_kwargs)
)

In [ ]:
figure()
# plot:
line_train, = plot([], [], 'go-', label="average training loss")
# line_curr_val, = plot([],[], 'bo:', label="avrg current validation loss")
# line_best_val, = plot([], [], 'ko-', label="avrg best validation loss")
# plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
yscale('log')
gca().yaxis.set_minor_formatter(FormatStrFormatter("%.2g"))
ylabel("loss")
xlabel("#epoch")
legend(loc='lower left', fancybox=True, framealpha=0.5)

# start values:
best_parameters = optimizer_kwargs['num_parameters']
# best_val_loss = optimizer_kwargs['num_loss'](best_parameters, validation, no_annealing=True)
# best_val_loss /= len(validation)
training_loss = optimizer_kwargs['num_loss'](best_parameters, targets)#, no_annealing=True)
# training_loss /= len(targets)
last_improvement_epoch = opt.n_iter//n_batches

# add_point(line_best_val, last_improvement_epoch, best_val_loss)
# add_point(line_curr_val, last_improvement_epoch, best_val_loss)
add_point(line_train, last_improvement_epoch, training_loss)

In [ ]:
fig_target_distr = figure()
plot_distr_target(fig_target_distr)

In [ ]:
fig_model_distr = figure()
plot_distr_model(fig_model_distr, opt.wrt)

In [ ]:
fig, ax = subplots()
plot_fit(fig, ax)  #{_model['flat']:optimizer_kwargs['num_parameters']}

THESE ARE SOME OF THE IMPORTANT VALUES. For the targets logP it is clear the meanexpmap is the right one, and it works pretty well.
However for the variational lowerbound officially meanmap is correct, but it gives still impressively worse results
then the meanexpmap. (there is some additional bias due to the prior, but this is not decisive (even with n_data=1))

In [ ]:
logP_maximum_likelihood = target.loglikelihood.function()

In [ ]:
meanexpmap(logP_maximum_likelihood, targets)

In [ ]:
meanmap(logP_maximum_likelihood, targets)

In [ ]:
logP_variational = model.logP.function()

In [ ]:
meanexpmap(logP_variational, targets)

In [ ]:
meanmap(logP_variational, targets)

---

In [ ]:
human_readable_parameters = model['parameters'] + model['parameters_positive']
f = theano.function([_model['flat']], human_readable_parameters)
old_num_params = f(opt.wrt)
        
with ignored(KeyboardInterrupt):
    for info in every(n_batches, opt):  #every(n_batches, opt):
        current_epoch = info['n_iter']//n_batches
        # visualize training loss for comparison:
        training_loss = optimizer_kwargs['num_loss'](opt.wrt, targets)#, no_annealing=True)
#         training_loss /= len(targets)
        add_point(line_train, current_epoch, training_loss)
        
        
        num_params = f(opt.wrt)
        print "jacobiannames", str(_model['flat'])
        print "jacobian", optimizer_kwargs['num_jacobian'](opt.wrt, targets[:batch_size])
        diff_params = [n - o for n, o in izip(num_params, old_num_params)]
        print("params")
        pprint(dict(zip(map(str, human_readable_parameters), num_params)))
        print("diff")
        pprint(dict(zip(map(str, human_readable_parameters), diff_params)))
        print
        
        plot_distr(fig_model_distr, opt.wrt)
        plot_fit(fig, ax, {_model['flat']:opt.wrt}, time_delay=None)
        old_num_params = num_params

In [ ]:
%debug

---

In [ ]:
best_parameters = None
best_val_loss = inf
last_improvement_epoch = 1
for info in every(n_batches, opt):
    current_epoch = info['n_iter']//n_batches
    if current_epoch - last_improvement_epoch > 50:
        break
    # collect and visualize validation loss for choosing the best model
    val_loss = optimizer_kwargs['num_loss'](opt.wrt, training, no_annealing=True)
    if val_loss < best_val_loss:
        last_improvement_epoch = current_epoch
        best_parameters = opt.wrt
        best_val_loss = val_loss
    
    print zip(model['flat'].name.split(':'), climin_kwargs['fprime'](opt.wrt, training[:100], no_annealing=True))
    plot_fit(fig, ax, givens={model['flat']: opt.wrt}, time_delay=None)
    print best_val_loss,

In [ ]:
%debug

In [ ]:
optimizer_kwargs['flat']

In [ ]:
%timeit optimizer_kwargs['num_jacobian'](best_parameters, targets[:100])

In [ ]:
stats = np.array([optimizer_kwargs['num_jacobian'](best_parameters, targets[:100]) for _ in range(1000)])

In [ ]:
stats[:100]

In [ ]:
plot(stats.T)
pass

In [ ]:
stats.mean(0)

In [ ]:
stats.var(0)

In [ ]:
opt.n_iter

In [ ]:
model.u.eval({model['flat']: best_parameters})

In [ ]:
theano.function([model['flat']], 
                [model.b, target.b,
                 model.w, target.w,
                 model._u, target._u, 
                 param_model_base.mean, 
                 param_target_base.mean, 
                 param_model_base.var, param_target_base.var,
                 data_model.var],
                mode="FAST_COMPILE")(best_parameters)

In [ ]:
best_val_loss

Using the default scipy optimizer, one can see very good, that the problem is ill-defined in that the maximum likelihood solution is a ill-formed solution.